In [1]:
import os
import geopandas as gpd
import pandas as pd
import ee
import geemap
import env_data_acq_func as isb
import segment_attributes_func as gsf
#import gee_data_acq as gda

In [2]:
# Authenticate to Earth Engine
ee.Authenticate()

True

In [2]:
# Authenticate to Earth Engine
#ee.Authenticate()
ee.Initialize()

In [1]:
# Create a map using geemap
#m = geemap.Map()
#m

In [3]:
folder_directory = 'D:/!!Research/!!!Data/ArcGIS_Projects'
raster_working = f'{folder_directory}/quick_new_poly/files_for_python'
bands_csv_name = 'images_and_bands.csv'
images_and_bands = pd.read_csv(f'{raster_working}/{bands_csv_name}')
images_and_bands.head()

,collection,collection_loc,bands,band_rename,base_re,unit,resample,resample_res,resample_method
0,clay_0_5,projects/sat-io/open-datasets/polaris/clay_mea...,b1,clay_0_5,30,m,True,50,bilinear
1,clay_5_15,projects/sat-io/open-datasets/polaris/clay_mea...,b1,clay_5_15,30,m,True,50,bilinear
2,clay_15_30,projects/sat-io/open-datasets/polaris/clay_mea...,b1,clay_15_30,30,m,True,50,bilinear
3,clay_30_60,projects/sat-io/open-datasets/polaris/clay_mea...,b1,clay_30_60,30,m,True,50,bilinear
4,clay_60_100,projects/sat-io/open-datasets/polaris/clay_mea...,b1,clay_60_100,30,m,True,50,bilinear


In [4]:
#shrubs_states = ['california', 'kansas', 'new_mexico', 'nevada', 'texas']
#sts_abbvs = ['CA', 'KS', 'NM', 'NV', 'TX']
sts_abbrv = ['TX', 'NM']

In [5]:
other_bands = []

elevation = ee.Image('USGS/3DEP/10m').select(['elevation'])
other_bands.append(elevation)
slope = ee.Terrain.slope(elevation).select(['slope']).rename(['slope'])
other_bands.append(slope)
aspect = ee.Terrain.aspect(elevation).select(['aspect']).rename(['aspect'])
other_bands.append(aspect)

nhd_area_tx = ee.FeatureCollection(f"projects/sat-io/open-datasets/NHD/NHD_{sts_abbrv[0]}/NHDArea")
nhd_flowline_tx = ee.FeatureCollection(f"projects/sat-io/open-datasets/NHD/NHD_{sts_abbrv[0]}/NHDFlowline")
nhd_tx = nhd_area_tx.merge(nhd_flowline_tx)
nhd_area_nm = ee.FeatureCollection(f"projects/sat-io/open-datasets/NHD/NHD_{sts_abbrv[1]}/NHDArea")
nhd_flowline_nm = ee.FeatureCollection(f"projects/sat-io/open-datasets/NHD/NHD_{sts_abbrv[1]}/NHDFlowline")
nhd_nm = nhd_area_nm.merge(nhd_flowline_nm)
nhd = nhd_tx.merge(nhd_nm)
dist_nhd = nhd.distance(searchRadius=10000, maxError=50) #originally searchRadius = 50000
#dist_nhd = dist_nhd.clip(ee_items[num]) #change clip area
dist_nhd = dist_nhd.select(['distance'])
dist_nhd = dist_nhd.rename('dist_drain')
other_bands.append(dist_nhd)


#roads = ee.FeatureCollection('TIGER/2016/Roads')
#dist_roads = roads.distance(searchRadius=50000, maxError=50)
#dist_roads = dist_roads.select(['distance'])
#dist_roads = dist_roads.rename('dist_road')
#other_bands.append(dist_roads)

print(other_bands)

[<ee.image.Image object at 0x000001AF923A8D70>, <ee.image.Image object at 0x000001AF93F442C0>, <ee.image.Image object at 0x000001AF93EF5C40>, <ee.image.Image object at 0x000001AF93F98EC0>]


In [25]:
import re
#roi = ee.FeatureCollection('projects/ee-jdawsey/assets/acc_points_wCoords')
shp_list = []
for file in os.listdir(shp_dir):
    shps = []
    regex_gcs = re.compile(r'[0-9]_gcs+')
    if file.endswith('.shp'):
        shps.append(f'{shp_dir}/{file}')
    for shp in shps:
        if (regex_gcs.search(shp) == None) == True:
            shp_list.append(geemap.shp_to_ee(shp))
        else:
            pass
first_shape = shp_list[0]
shp_list.pop(0)
for shp in shp_list:
    first_shape.merge(shp)


############# need to figure out how to prevent it being a collection of collections
roi = first_shape

In [7]:
shp_directory = 'D:/!!Research/!!!Data/ArcGIS_Projects/'
points_dir = f'{shp_directory}/useful_gdbs_and_shps/acc_points/'
#points_name = '....shp'
points_path = f'{points_dir}acc_point_wCoord.shp'
roi = geemap.shp_to_ee(points_path)

In [ ]:
# Define a time period
start_date = '2016-06-01'
end_date = '2016-09-30'

# Load Landsat 8 image collection
landsat_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(roi).filterDate(start_date, end_date).filterMetadata('CLOUD_COVER', 'less_than', 20)

# Apply normalization to the collection
normalized_collection = landsat_collection.map(normalize).mosaic()
normalized_collection.bandNames().getInfo()

# Mosaic the normalized collection
#ls_mosaic = normalized_collection.mosaic().clip(roi)

#ls_mosaic.bandNames().getInfo()
"""
#ls_bands = []
for band in ls_mosaic.bandNames().getInfo():
    other_bands.append(ls_mosaic.select(['band']))

#other_bands.append(ls_mosaic)
print(other_bands)
"""

In [ ]:
Map = geemap.Map()
Map

vis_param_gauss = {'bands' : ['b1'], 
               'min' : 0, 
               'max' : 1,
               'gamma' : 1}

Map.addLayer(ls_mosaic, vis_param_gauss)

In [6]:
image_stack = isb.build_image_stack(images_and_bands, other_bands)
print(image_stack.bandNames().getInfo())

['clay_0_5', 'clay_5_15', 'clay_15_30', 'clay_30_60', 'clay_60_100', 'clay_100_200', 'sand_0_5', 'sand_5_15', 'sand_15_30', 'sand_30_60', 'sand_60_100', 'sand_100_200', 'silt_0_5', 'silt_5_15', 'silt_15_30', 'silt_30_60', 'silt_60_100', 'silt_100_200', 'bd_0_5', 'bd_5_15', 'bd_15_30', 'bd_30_60', 'bd_60_100', 'bd_100_200', 'om_0_5', 'om_5_15', 'om_15_30', 'om_30_60', 'om_60_100', 'om_100_200', 'ph_0_5', 'ph_5_15', 'ph_15_30', 'ph_30_60', 'ph_60_100', 'ph_100_200', 'ksat_0_5', 'ksat_5_15', 'ksat_15_30', 'ksat_30_60', 'ksat_60_100', 'ksat_100_200', 'theta_r_0_5', 'theta_r_5_15', 'theta_r_15_30', 'theta_r_30_60', 'theta_r_60_100', 'theta_r_100_200', 'theta_s_0_5', 'theta_s_5_15', 'theta_s_15_30', 'theta_s_30_60', 'theta_s_60_100', 'theta_s_100_200', 'caco3', 'cec', 'ec', 'sar', 'kw_025', 'resdept', 'soil_depth', 'wind_erodibility', 'rf_025', 'water_storage', 'ann_rain', 'temp_range', 'elevation', 'slope', 'aspect', 'dist_drain']


In [7]:
#shp_dir = f'{folder_directory}shp_file'
# Don't need to repeate the folder directory in an fstring, just need to tack on the
# additional directories that lead to the folder you want to store the data in.
# e.g. 'test_mesq_prediction_points/acc_points_env_data/env_data_no_ls'
data_folder = 'D:/!!Reseach/!!!Data/shrubmap_modification_tests/points_data'
shp_dir = 'D:/!!Research/!!!Data/shrubmap_modification_tests/points_fishnet'


#shp_data = gda.PointEnvData(folder_directory, data_folder, image_stack, shp_dir)
isb.process_shp_files(folder_directory, data_folder, image_stack, shp_dir)

savi ((1 + 0.6) * (b("N") - b("R"))) / (b("N") + b("R") + 0.5)
endvi ((b("N") + b("G")) - (2 * b("B"))) / ((b("N") + b("G")) + (2 + b("B")))
Generating URL ...
Please wait ...
An error occurred while downloading.
Error extracting values to points: Expecting value: line 1 column 1 (char 0)
skipping gcs file
savi ((1 + 0.6) * (b("N") - b("R"))) / (b("N") + b("R") + 0.5)
endvi ((b("N") + b("G")) - (2 * b("B"))) / ((b("N") + b("G")) + (2 + b("B")))
Generating URL ...
Please wait ...
An error occurred while downloading.
Error extracting values to points: Expecting value: line 1 column 1 (char 0)
savi ((1 + 0.6) * (b("N") - b("R"))) / (b("N") + b("R") + 0.5)
endvi ((b("N") + b("G")) - (2 * b("B"))) / ((b("N") + b("G")) + (2 + b("B")))
Generating URL ...


KeyboardInterrupt: 